In [1]:
# Find conda environment
# /home/ahnaf/py3_env/pysyft381
# !echo $CONDA_PREFIX

import os
import matplotlib.pyplot as plt
import pickle, time, tqdm
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

#The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from data_utils import dataset_select, DataSubset  # Returns the train and test sets for the chosen dataset; dataset_select and class DataSubset
from data_dist import * # (Returns the dictionary of nodes/data partitions for both iid and nidd) )
# from DNN import * # (Returns Network, client update, aggregate)
from d2denvironment import *  #(returns cluster_set, graph, Laplacian and Eigenvalues)
from train_round import *
from DNN import *
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import matplotlib.pyplot as plt
from data_dist import *
from train_round import *
num_nodes = 10
num_rounds = 15
dataset = 'mnist'
traindata, testdata = dataset_select(dataset)
train_dist = data_iid(traindata, num_nodes)
test_dist = data_iid(testdata, num_nodes)

train_loader = torch.utils.data.DataLoader(traindata, batch_size = 32)
test_loader =torch.utils.data.DataLoader(testdata)
agg_model = Net(10, 1, dataset).cuda()
models = {node:Net(10, 1, dataset).cuda() for node in range(num_nodes)}
optimizers = {node:torch.optim.SGD(model.parameters(), lr = 0.1) for node, model in models.items()}

In [ ]:
nodeloss_list = {node:[] for node in range(num_nodes)}
nodeacc_list = {node:[] for node in range(num_nodes)}

aggloss_list = []
aggacc_list = []

for rnd in range(num_rounds):
    for node, model in models.items():
        model_updated = client_update(model, optimizers[node], DataLoader(DataSubset(traindata, train_dist, node), batch_size = 32), nodeloss_list[node], 2)

    aggregated_model = server_aggregate(models, list(range(num_nodes)))
    
    for node in range(num_nodes):
        aggloss, aggacc = test(aggregated_model, DataLoader(DataSubset(testdata, test_dist, node)))
        nodeloss, nodeacc = test(models[node], DataLoader(DataSubset(testdata, test_dist, node)))
        
        nodeloss_list[node].append(nodeloss)
        aggloss_list.append(aggloss)
        nodeacc_list[node].append(nodeacc)
        aggacc_list.append(aggacc)
        print(f'Accuracy for aggregated model for node{node} is {aggacc:0.5f}')
        print(f'Individual model accuracy for node{node} is {nodeacc:0.5f}')
        
        models[node].load_state_dict(aggregated_model.state_dict())

In [ ]:
for node in list(range(num_nodes)):
    plt.plot(nodeloss_list[node])

In [ ]:
for node in list(range(num_nodes)):
    plt.plot(nodeacc_list[node])

In [ ]:
plt.plot(aggacc_list)

In [ ]:
for node in range(num_nodes):
    nodeloss, nodeacc = test(aggregated_model, DataLoader(DataSubset(testdata, test_dist, node)))
    print(nodeloss, nodeacc)

In [ ]:
for node in range(num_nodes):
    nodeloss, nodeacc = test(models[0], DataLoader(DataSubset(testdata, test_dist, node)))
    print(nodeloss, nodeacc)

In [ ]:
for i in range(len(loss_list)):
    plt.plot(loss_list[i])

In [ ]:
for i in range(len(nodeacc_list)):
    plt.plot(aggacc_list[i])

In [ ]:
def aggregator(client_models, node_list):    
    base_model = copy.deepcopy(client_models[0])
    for k in base_model.state_dict().keys():
        base_model.state_dict()[k] = torch.stack([client_models[node].state_dict()[k].float() for node in node_list], 0).mean(0)
    return base_model

def model_checker(model1, model2):
    models_differ = 0
    for modeldata1, modeldata2 in zip(model1.state_dict().items(), model2.state_dict().items()):
        if torch.equal(modeldata1[1], modeldata2[1]):
            pass
        else:
            models_differ += 1
            if (modeldata1[0] ==  modeldata2[0]):
                print("Mismatch at ", modeldata1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match')

In [2]:
# Testing function
test_list = {node:[] for node in range(num_nodes)}
model_testing(models, testdata, test_dist, test_list)

/home/ahnaf/Fed_Learn/Divergence/data_utils.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)


Accuracy for node0 is 0.11400
Test loss for node0 is 2.31030
Accuracy for node1 is 0.09100
Test loss for node1 is 2.30342
Accuracy for node2 is 0.07800
Test loss for node2 is 2.30632
Accuracy for node3 is 0.11500
Test loss for node3 is 2.31341
Accuracy for node4 is 0.09100
Test loss for node4 is 2.30462
Accuracy for node5 is 0.05400
Test loss for node5 is 2.31443
Accuracy for node6 is 0.08200
Test loss for node6 is 2.30706
Accuracy for node7 is 0.10100
Test loss for node7 is 2.29795
Accuracy for node8 is 0.10200
Test loss for node8 is 2.31248
Accuracy for node9 is 0.08700
Test loss for node9 is 2.30878


In [8]:
!python Main_Fed.py -n 6 -r 1 -e 1 -c 2 -m iid

Starting update and aggregation for d2d_clus
The cluster configuration for d2d_clus is [array([1, 4, 5, 0]), array([4, 1, 3, 0])]
  0%|                                                     | 0/1 [00:00<?, ?it/s]/home/ahnaf/Fed_Learn/Divergence/data_utils.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)
100%|█████████████████████████████████████████████| 1/1 [00:28<00:00, 28.95s/it]
 Loss for node 0 in the round-epoch 0 is 0.30289
100%|█████████████████████████████████████████████| 1/1 [00:28<00:00, 28.87s/it]
 Loss for node 1 in the round-epoch 0 is 0.30422
100%|█████████████████████████████████████████████| 1/1 [00:28<00:00, 28.27s/it]
 Loss for node 2 in the round-epoch 0 is 0.30237
100%|█████████████████████████████████████████████| 1/1 [00:27<00:00, 27.94s/it]
 Loss for node 3 in th